In [ ]:
# Дан файл с некоторыми данными. Формат файла – произвольный. 
# Считайте данные из файла и запишите их в Mongo. 
# Реализуйте и выполните следующие запросы:
#   вывод первых 10 записей, отсортированных по убыванию по полю salary;
#   вывод первых 15 записей, отфильтрованных по предикату age < 30, отсортировать по убыванию по полю salary
#   вывод первых 10 записей, отфильтрованных по сложному предикату: 
#          (записи только из произвольного города, записи только из трех произвольно взятых профессий), 
#          отсортировать по возрастанию по полю age
#   вывод количества записей, получаемых в результате следующей фильтрации 
#          (age в произвольном диапазоне, year в [2019,2022], 50 000 < salary <= 75 000 || 125 000 < salary < 150 000). 
#   здесь и везде предполагаем вывод в JSON.

In [3]:
!pip install pymongo

In [60]:
import json
import msgpack
from pymongo import MongoClient, collection
from bson import ObjectId

In [61]:
with open('task_1_item.msgpack', 'rb') as file:
        msgpack_data = msgpack.unpack(file)
msgpack_data[0:3]        

[{'job': 'Психолог',
  'salary': 197587,
  'id': 9164940,
  'city': 'Душанбе',
  'year': 2020,
  'age': 49},
 {'job': 'Инженер',
  'salary': 132598,
  'id': 9460738,
  'city': 'Осера',
  'year': 2021,
  'age': 49},
 {'job': 'Повар',
  'salary': 67667,
  'id': 8333764,
  'city': 'Минск',
  'year': 2003,
  'age': 32}]

In [62]:
def read_data_from_file(file_path):
    with open(file_path, 'rb') as file:
        data = msgpack.unpack(file)
    return data

In [63]:
def write_data_to_mongodb(data, host, port, db_name, collection_name):
    client = MongoClient(host, port)
    db = client[db_name]
    collection = db[collection_name]
    collection.insert_many(data)
    client.close()

In [64]:
def get_top_10_records_sorted_by_salary(collection):
    result = collection.find().sort('salary', -1).limit(10)
    return list(result)

In [65]:
def get_top_15_records_filtered_by_age(collection):
    result = collection.find({'age': {'$lt': 30}}).sort('salary', -1).limit(15)
    return list(result)

In [66]:
def get_top_10_records_filtered_by_city_and_jobs(collection, city, jobs):
    result = collection.find({'city': city, 'job': {'$in': jobs}}).sort('age', 1).limit(10)
    return list(result)

In [67]:
def get_records_count_filtered_by_conditions(collection, age_range, year_range, salary_range):
    result = collection.count_documents({
        '$and': [
            {'age': {'$gt': age_range[0], '$lt': age_range[1]}},
            {'year': {'$gte': year_range[0], '$lte': year_range[1]}},
            {'$or': [
                {'salary': {'$gt': salary_range[0], '$lte': salary_range[1]}},
                {'salary': {'$gt': salary_range[2], '$lt': salary_range[3]}}
            ]}
        ]
    })
    return result

In [85]:
def default_json_encoder(obj):
    if isinstance(obj, ObjectId):
        return str(obj)
    raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")

In [86]:
def write_result_to_json(result, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(result, file, indent=4, ensure_ascii=False, default=default_json_encoder)

In [87]:
# Параметры
file_path = 'task_1_item.msgpack'
host = 'localhost'
port = 27017
db_name = 'mydatabase'
collection_name = 'mycollection'
city = 'Москва'
jobs = ['Повар', 'Инженер', 'Учитель']
age_range = (30, 50)
year_range = (2019, 2022)
salary_range = (50000, 75000, 125000, 150000)

In [88]:
# Чтение данных из файла
data = read_data_from_file(file_path)

In [89]:
# Запись данных в MongoDB
write_data_to_mongodb(data, host, port, db_name, collection_name)

In [90]:
# Подключение к MongoDB и получение коллекции
client = MongoClient(host, port)
db = client[db_name]
collection = db[collection_name]

In [91]:
# Вывод первых 10 записей, отсортированных по убыванию по полю salary
result1 = get_top_10_records_sorted_by_salary(collection)
write_result_to_json(result1, 'result1_1.json')

In [92]:
# Вывод первых 15 записей, отфильтрованных по предикату age < 30, отсортировать по убыванию по полю salary
result2 = get_top_15_records_filtered_by_age(collection)
write_result_to_json(result2, 'result2_1.json')

In [93]:
# Вывод первых 10 записей, отфильтрованных по сложному предикату
result3 = get_top_10_records_filtered_by_city_and_jobs(collection, city, jobs)
write_result_to_json(result3, 'result3_1.json')

In [94]:
# Вывод количества записей, получаемых в результате следующей фильтрации
result4 = get_records_count_filtered_by_conditions(collection, age_range, year_range, salary_range)
write_result_to_json(result4, 'result4_1.json')

In [95]:
# Закрытие соединения с MongoDB
client.close()